In [5]:
import pandas as pd
import requests
import time
from tqdm import tqdm
from pprint import pprint

In [9]:
import pandas as pd

wallets = pd.read_csv('wallet_id.csv')
wallets.columns = ['wallet_id']

# View first 5 wallet IDs
print(wallets.head())                     # Shows full DataFrame rows
print(wallets['wallet_id'].head())       # Shows just the 'wallet_id' column

                                    wallet_id
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1  0x06b51c6882b27cb05e712185531c1f74996dd988
2  0x0795732aacc448030ef374374eaae57d2965c16c
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
0    0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1    0x06b51c6882b27cb05e712185531c1f74996dd988
2    0x0795732aacc448030ef374374eaae57d2965c16c
3    0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4    0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Name: wallet_id, dtype: object


In [10]:
# === STEP 2: Setup Graph API ===
API_KEY = "73d7e0d853d2b742963509843fe08fff"
SUBGRAPH_ID = "AwoxEZbiWLvv6e3QdvdMZw4WDURdGbvPfHmZRc8Dpfz9"
url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

In [12]:
# === STEP 3: Define GraphQL Query Template ===
def build_query(wallet):
    return {
        "query": f"""
        {{
          account(id: "{wallet.lower()}") {{
            id
            positionCount
            openPositionCount
            closedPositionCount

            depositCount
            deposits {{ id amount blockNumber }}
            
            withdrawCount
            withdraws {{ id amount blockNumber }}
            
            borrowCount
            borrows {{ id amount blockNumber }}
            
            repayCount
            repays {{ id amount blockNumber }}
            
            liquidateCount
            liquidates {{ id amount blockNumber }}
            
            liquidationCount
            liquidations {{ id amount blockNumber }}
            
            transferredCount
            transfers {{ id amount blockNumber }}
            
            receivedCount
            receives {{ id amount blockNumber }}
            
            flashloanCount
            flashloans {{ id amount blockNumber }}
            
            rewardsClaimedUSD
          }}
        }}
        """
    }


In [13]:
results = {}
for wallet in tqdm(wallets['wallet_id']):
    success = False
    attempts = 0
    while not success and attempts < 3:
        try:
            response = requests.post(url, json=build_query(wallet), timeout=15)
            if response.status_code == 200:
                results[wallet] = response.json()
                success = True
            else:
                print(f"Error for {wallet}: Status Code {response.status_code}")
                print(response.text)
                time.sleep(2)
        except Exception as e:
            print(f"Error processing {wallet}: {e}")
            time.sleep(2)
            attempts += 1
    time.sleep(1) 

100%|██████████| 103/103 [03:56<00:00,  2.30s/it]


In [14]:

# Optionally: Save results to JSON
import json
with open('wallet_query_results.json', 'w') as f:
    json.dump(results, f, indent=2)

In [31]:
import requests
import json

# Set values
API_KEY = "73d7e0d853d2b742963509843fe08fff"
SUBGRAPH_ID = "AwoxEZbiWLvv6e3QdvdMZw4WDURdGbvPfHmZRc8Dpfz9"
BASE_URL = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

wallet_address = "0x0039f22efb07a647557c7c5d17854cfd6d489ef3".lower()

query = {
    "query": f"""
    {{
      account(id: "{wallet_address}") {{
        id
        deposits {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        withdraws {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        borrows {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        repays {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        liquidates {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        liquidations {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        flashloans {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
      }}
    }}
    """
}

# Make the request
response = requests.post(BASE_URL, json=query)

if response.status_code == 200:
    result = response.json()
    if result["data"]["account"]:
        print(json.dumps(result["data"]["account"], indent=2))
    else:
        print("⚠️ No transaction data found for this wallet.")
else:
    print(f"❌ Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

{
  "borrows": [],
  "deposits": [
    {
      "amount": "10000000000000000000",
      "asset": {
        "symbol": "WETH"
      },
      "id": "0xc399ce8e7581dbd898c1e07dd64dd55105c6aab249a9967051dc60cd3acf35542000000000000000",
      "timestamp": "1662508766"
    }
  ],
  "flashloans": [],
  "id": "0x0039f22efb07a647557c7c5d17854cfd6d489ef3",
  "liquidates": [],
  "liquidations": [],
  "repays": [],
  "withdraws": [
    {
      "amount": "10000000000000000000",
      "asset": {
        "symbol": "WETH"
      },
      "id": "0x023e7a3334e06129170d0fdaa281514ad5a2a65f3fd9d0e9adac70b82e6dc4c75802000001000000",
      "timestamp": "1662508878"
    }
  ]
}


In [ ]:
import requests
import json

# Config
API_KEY = "73d7e0d853d2b742963509843fe08fff"  # Replace with your Graph Studio API key
SUBGRAPH_ID_V2 = "4TbqVA8p2DoBd5qDbPMwmDZv3CsJjWtxo8nVSqF2tA9a"
V2_URL = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID_V2}"

wallet_address = "0x0039f22efb07a647557c7c5d17854cfd6d489ef3".lower()

query = {
    "query": f"""
    {{
      account(id: "{wallet_address}") {{
        id
        deposits {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        withdraws {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        borrows {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        repays {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        liquidates {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        liquidations {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
        flashloans {{
          id
          amount
          asset {{ symbol }}
          timestamp
        }}
      }}
    }}
    """
}

# Make the request
response = requests.post(BASE_URL, json=query)

if response.status_code == 200:
    result = response.json()
    if result["data"]["account"]:
        print(json.dumps(result["data"]["account"], indent=2))
    else:
        print("⚠️ No transaction data found for this wallet.")
else:
    print(f"❌ Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

{
  "borrows": [],
  "deposits": [
    {
      "amount": "10000000000000000000",
      "asset": {
        "symbol": "WETH"
      },
      "id": "0xc399ce8e7581dbd898c1e07dd64dd55105c6aab249a9967051dc60cd3acf35542000000000000000",
      "timestamp": "1662508766"
    }
  ],
  "flashloans": [],
  "id": "0x0039f22efb07a647557c7c5d17854cfd6d489ef3",
  "liquidates": [],
  "liquidations": [],
  "repays": [],
  "withdraws": [
    {
      "amount": "10000000000000000000",
      "asset": {
        "symbol": "WETH"
      },
      "id": "0x023e7a3334e06129170d0fdaa281514ad5a2a65f3fd9d0e9adac70b82e6dc4c75802000001000000",
      "timestamp": "1662508878"
    }
  ]
}


In [ ]:
import requests
import json
from pprint import pprint

API_KEY = "73d7e0d853d2b742963509843fe08fff"

# Subgraph endpoints
AAVE_V2_ID = "78vjx5nN57gJDJCGYafxkGNKQSEFfTNo4AkNBcRQZV2R"
COMPOUND_V3_ID = "5MjRndNWGhqvNX7chUYLQDnvEgc8DaH8eisEkcJt71SR"

def query_subgraph(subgraph_id, wallet):
    url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{subgraph_id}"
    gql = {
        "query": f"""
        {{
          userTransactions(where: {{ user: "{wallet}" }}) {{
            id
            timestamp
            txHash
            action
            ... on Deposit {{
              amount
              reserve {{ symbol }}
              assetPriceUSD
            }}
            ... on Borrow {{
              amount
              reserve {{ symbol }}
              assetPriceUSD
            }}
            ... on Repay {{
              amount
              reserve {{ symbol }}
              assetPriceUSD
            }}
            ... on Withdraw {{
              amount
              reserve {{ symbol }}
              assetPriceUSD
            }}
            ... on Liquidation {{
              collateralAmount
              principalAmount
              reserve {{ symbol }}
              assetPriceUSD
            }}
            ... on FlashLoan {{
              amount
              asset {{ symbol }}
              feeUSD
            }}
          }}
        }}
        """
    }
    resp = requests.post(url, json=gql)
    resp.raise_for_status()
    return resp.json().get("data", {}).get("userTransactions", [])

# Test using your wallet
wallet = "0x0039f22efb07a647557c7c5d17854cfd6d489ef3".lower()

aave_txns = query_subgraph(AAVE_V2_ID, wallet)
compound_txns = query_subgraph(COMPOUND_V3_ID, wallet)

print("Aave V2 Transactions:")
pprint(aave_txns[:5])
print("\nCompound V3 Transactions:")
pprint(compound_txns[:5])